# Support vector machine (SVM) model


A toy SVM model using a mini dataset

## Import packages

In [1]:
import quadprog
import numpy as np

def quadprog_solve_qp(P, q, G=None, h=None, A=None, b=None):
    qp_G = .5 * (P + P.T)   # make sure P is symmetric
    qp_a = -q
    if A is not None:
        qp_C = -np.vstack([A, G]).T
        qp_b = -np.hstack([b, h])
        meq = A.shape[0]
    else:  # no equality constraint
        qp_C = -G.T
        qp_b = -h
        meq = 0
    return quadprog.solve_qp(qp_G, qp_a, qp_C, qp_b, meq)[0]

# Toy data
X = np.array([
    [0, 0],
    [2, 0],
    [0, 2],
    [3, 3],
    [4, 4]
])
Y = np.array([-1, -1, -1, 1, 1])

# Task
Buid an SVM model on the toy dataset: 

\begin{equation}
    x^{(1)} = (0,0),\ y^{(1)}=-1\\
    x^{(2)} = (2,0),\ y^{(2)}=-1\\
    x^{(3)} = (0,2),\ y^{(3)}=-1\\
    x^{(4)} = (3,3),\ y^{(4)}=1\\
    x^{(5)} = (4,4),\ y^{(5)}=1\\
\end{equation}

Solve the quadratic programming (QP) problem as the following form:

\begin{equation}
    \min_{\alpha}\big( \frac{1}{2}\alpha^{T}Q\alpha - (\textbf{1})^{T}\alpha \big) \\
    \text{subject to: } y^{T}\alpha=0,\ \alpha\geq 0
\end{equation}

The quadprog package by defaualt solves the QP as this form:

\begin{equation}
    \min_{x}\big( \frac{1}{2}x^{T}Px + q^{T}x \big) \\
    \text{subject to: } Gx\leq h,\ Ax = b
\end{equation}

Therefore, in order to use quadprog, we need to establish the responding relationships between variables: 
$P=Q$, $q = -(\textbf{1})^{T}$, $G = -(\textbf{1})^{T}$, $h=(\textbf{0})^{T}$, $A=y^T$, $b=(\textbf{0})^{T}$


## Task 1: Compute matrix $Q$


First need to use, $x^{(i)}$ and $y^{(i)}$ to compute matrix $Q$:

\begin{equation}
    Q = \begin{bmatrix}
    y^{(1)}y^{(1)}x^{(1)T}x^{(1)} & y^{(1)}y^{(2)}x^{(1)T}x^{(2)} & \dots & y^{(1)}y^{(5)}x^{(1)T}x^{(5)} \\
    y^{(2)}y^{(1)}x^{(2)T}x^{(1)} & y^{(2)}y^{(2)}x^{(2)T}x^{(2)} & \dots & y^{(2)}y^{(5)}x^{(2)T}x^{(5)} \\
    \vdots & \vdots & \ddots & \vdots \\
    y^{(5)}y^{(1)}x^{(5)T}x^{(1)} & y^{(5)}y^{(2)}x^{(5)T}x^{(2)} & \dots & y^{(5)}y^{(5)}x^{(5)T}x^{(5)} \\
    \end{bmatrix}
\end{equation}


In [2]:
Q = np.zeros((5, 5))

for i in range(Q.shape[0]):
    for j in range(Q.shape[1]):
        # Use the ith and jth examples in X and Y to compute Q_ij
        Q[i, j] = Y[i]*Y[j]*np.dot(X[i],X[j])

print('Q = ', Q)

Q =  [[ 0.  0.  0.  0.  0.]
 [ 0.  4.  0. -6. -8.]
 [ 0.  0.  4. -6. -8.]
 [ 0. -6. -6. 18. 24.]
 [ 0. -8. -8. 24. 32.]]


## Task 2: Computer other variables

Formulas: $P=Q$, $q = -(\textbf{1})^{T}$, $G = -(\textbf{1})^{T}$, $h=(\textbf{0})^{T}$, $A=y^T$, $b=(\textbf{0})^{T}$

In [3]:
P = Q + np.eye(5)*1e-5 # To solve the non-positive finite issue
q = -(np.ones(5)).T
G = -(np.eye(5)).T
h = (np.zeros(5)).T

A = Y.reshape((1,5))

b = (np.zeros(1)).T

print('q = ', q)
print('G = ', G)
print('h = ', h)
print('b = ', b)

q =  [-1. -1. -1. -1. -1.]
G =  [[-1. -0. -0. -0. -0.]
 [-0. -1. -0. -0. -0.]
 [-0. -0. -1. -0. -0.]
 [-0. -0. -0. -1. -0.]
 [-0. -0. -0. -0. -1.]]
h =  [0. 0. 0. 0. 0.]
b =  [0.]


## Task 3: Call quadprog

In [4]:
solution = quadprog_solve_qp(P, q, G, h, A, b)

print('solution = ', solution)
print('The support vectors are: ', X[solution > 0, ])

solution =  [0.         0.12499977 0.12499977 0.24999953 0.        ]
The support vectors are:  [[2 0]
 [0 2]
 [3 3]]


## Task 4: Solve the decision boundary

Using the support vectors to solve the $w$ and $b$ in the decision boundary $w^Tx+b=0$. Using the property that a support vector $x^{(k)}$ must satistify $y^{(k)}(w^Tx^{(k)}+b) = 1$.

#### Step 1: Filtered alpha values greater than zero for support vectors from solution
#### Step 2: Took x1 and x2 as first and second column of X respectively
#### Step 3: Took Y values for support vectors
#### Step 4: Using formula for w (summation of product for i values of aplha,y and x), calculated w1 and w2 
#### Step 5: Calculated value of b by substituting value of w1, x1 and y1 for single instance

In [5]:
alpha = solution[solution > 0]
x = X[solution > 0, ]
x1 = x[:,0]
x2 = x[:,1]
y = Y[1:4]

w1 = round((alpha * y).dot(x1),2)
w2 = round((alpha * y).dot(x2),2)
b = round(((1/y[0]) - (w1.T * x1[0])),2)

print('w1 = ', w1)
print('w2 = ', w2)
print('b = ', b)

w1 =  0.5
w2 =  0.5
b =  -2.0
